In [ ]:
# mount gg drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans, DBSCAN
from sklearn.metrics import silhouette_score
from sklearn.neighbors import NearestNeighbors

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/BI/Global_Superstore2.csv', encoding='latin1')
df


In [ ]:
df.info()
df.describe()

In [ ]:
df.isnull().sum()

# **Data Preprocessing**

In [ ]:
df.drop("Postal Code",axis=1,inplace=True)

In [ ]:
def check_duplicates(df):
    print("Duplicates :",df.duplicated().sum())
check_duplicates(df)

In [ ]:
import numpy as np
# show the missing values in dataset with ratio
def missing_values_tabl(df):

    na_columns = [col for col in df.columns if df[col].isnull().sum() > 0]
    n_miss = df[na_columns].isnull().sum().sort_values(ascending=False)
    ratio = (df[na_columns].isnull().sum() / df.shape[0] * 100).sort_values(ascending=False)
    missing_df = pd.concat([n_miss, np.round(ratio,2)], axis=1, keys=['n_miss', 'ratio'])
    missing_df = pd.DataFrame(missing_df)
    return missing_df

missing_values_tabl(df)

In [ ]:
# converting object to datetime
df['Order Date'] = pd.to_datetime(df['Order Date'], dayfirst=True, errors='coerce')
df['Ship Date']  = pd.to_datetime(df['Ship Date'],  dayfirst=True, errors='coerce')

In [ ]:
# creating new features
df["order year"]=df["Order Date"].dt.year
df["order_month"]=df["Order Date"].dt.month
df["ship year"]=df["Ship Date"].dt.year
df["ship_month"]=df["Ship Date"].dt.month
# unit price
df["unit_price"]=df["Sales"]/df["Quantity"]
# rename columns
df.rename(columns={"Order Date":"order_date","Ship Date":"ship_date","Order Priority":"order_priority"},inplace=True)

# **Exploratory Data Analysis (EDA)**

In [ ]:
# Order Priority vs Ship Mode
plt.figure(figsize=(10, 6))
sns.countplot(data=df, x='Ship Mode', hue='order_priority')
plt.title('Order Priority vs Ship Mode')
plt.xlabel('Ship Mode')
plt.ylabel('Count')
plt.legend(title='Order Priority')
plt.show()

# Total Sales by Market
market_sales = df.groupby('Market')['Sales'].sum().sort_values(ascending=False)
plt.figure(figsize=(10, 6))
market_sales.plot(kind='bar')
plt.title('Total Sales by Market')
plt.xlabel('Market')
plt.ylabel('Total Sales')
plt.show()

# Total Profit by Segment
segment_profit = df.groupby('Segment')['Profit'].sum().sort_values(ascending=False)
plt.figure(figsize=(10, 6))
segment_profit.plot(kind='bar')
plt.title('Total Profit by Segment')
plt.xlabel('Segment')
plt.ylabel('Total Profit')
plt.show()

# Order Count by Region
region_order_count = df['Region'].value_counts()
plt.figure(figsize=(10, 6))
region_order_count.plot(kind='bar')
plt.title('Order Count by Region')
plt.xlabel('Region')
plt.ylabel('Order Count')
plt.show()

# Discount vs Profit
plt.figure(figsize=(10, 6))
sns.scatterplot(data=df, x='Discount', y='Profit')
plt.title('Discount vs Profit')
plt.xlabel('Discount')
plt.ylabel('Profit')
plt.show()

# Discount vs Sales
plt.figure(figsize=(10, 6))
sns.scatterplot(data=df, x='Discount', y='Sales')
plt.title('Discount vs Sales')
plt.xlabel('Discount')
plt.ylabel('Sales')
plt.show()

# Discount vs unit_price
plt.figure(figsize=(10, 6))
sns.scatterplot(data=df, x='Discount', y='unit_price')
plt.title('Discount vs unit_price')
plt.xlabel('Discount')
plt.ylabel('unit_price')
plt.show()

In [ ]:
discount_values = [0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85]
df_filtered = df[df['Discount'].isin(discount_values)]

# Total Sales by Discount
discount_sales = df_filtered.groupby('Discount')['Sales'].sum()
plt.figure(figsize=(10, 6))
discount_sales.plot(kind='bar', color='skyblue')
plt.title('Total Sales by Discount')
plt.xlabel('Discount')
plt.ylabel('Total Sales')
plt.xticks(rotation=45)
plt.show()

# Total Profit by Discount
discount_profit = df_filtered.groupby('Discount')['Profit'].sum()
plt.figure(figsize=(10, 6))
discount_profit.plot(kind='bar', color='salmon')
plt.title('Total Profit by Discount')
plt.xlabel('Discount')
plt.ylabel('Total Profit')
plt.xticks(rotation=45)
plt.show()

# **K-Means Clustering ( Phân cụm khách hàng )**

In [ ]:
# Tính toán RFM cho phân cụm
rfm = df.groupby('Customer ID').agg({
    'order_date': lambda x: (df['order_date'].max() - x.max()).days,
    'Order ID': 'nunique',
    'Sales': 'sum'
}).reset_index()
rfm.columns = ['Customer ID', 'Recency', 'Frequency', 'Monetary']

# Kiểm tra xem rfm có NaN không
print(rfm[['Recency','Frequency','Monetary']].isnull().sum())

In [ ]:
# Tạo DataFrame sạch: drop những dòng có Recency NaN
rfm = rfm.dropna(subset=['Recency','Frequency','Monetary']).reset_index(drop=True)
# Chuẩn hóa dữ liệu khách hàng
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
rfm_scaled = scaler.fit_transform(rfm[['Recency','Frequency','Monetary']])

In [ ]:
# Silhouette scores for RFM
sil_scores = []
for k in range(2,8):
    km = KMeans(n_clusters=k, random_state=42).fit(rfm_scaled)
    sil_scores.append(silhouette_score(rfm_scaled, km.labels_))
    sil = silhouette_score(rfm_scaled, km.labels_)
    print(f"RFM - k={k}: Silhouette Score = {sil:.3f}")


# Plot silhouette analysis for RFM
plt.figure()
plt.plot(range(2,8), sil_scores, 'o-')
plt.xlabel('k clusters')
plt.ylabel('Silhouette Score')
plt.title('Silhouette Analysis cho RFM')
plt.show()


In [ ]:
# Nhận thấy K=3 là điểm tối ưu để Silhouett score lớn nhất
k_optimal = 3
kmeans = KMeans(n_clusters=k_optimal, random_state=42)
rfm['Cluster'] = kmeans.fit_predict(rfm_scaled)

# Silhouette Score KMeans
print("Silhouette Score KMeans:", silhouette_score(rfm_scaled, rfm['Cluster']))

In [ ]:
# Đếm số lượng khách hàng trong mỗi cụm
counts = rfm['Cluster'].value_counts().sort_index()
# Phân tích kết quả trung bình các đặc trưng theo cụm
cluster_summary = rfm.groupby('Cluster')[['Recency', 'Frequency', 'Monetary']].mean().round(2)
print("\n Trung bình RFM theo từng cụm:\n", cluster_summary)
# Số lượng sản phẩm trong mỗi cụm
print("\n Số lượng khách hàng trong mỗi cụm:\n", counts)

In [ ]:
# Vẽ biểu đồ trực quan
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
pcs = pca.fit_transform(rfm_scaled)
df_pca = pd.DataFrame(pcs, columns=['PC1','PC2'])
df_pca['Cluster'] = rfm['Cluster']
sns.scatterplot(data=df_pca, x='PC1', y='PC2', hue='Cluster', palette='tab10')
plt.title("RFM clusters in 2D PCA space")
plt.show()
sns.pairplot(rfm, hue='Cluster', vars=['Recency', 'Frequency', 'Monetary'], palette='tab10')
plt.suptitle("Phân cụm khách hàng theo RFM", y=1.02)
plt.show()

# **K-Means Clustering ( Phân cụm sản phẩm )**

In [ ]:
# Tạo feature cho product
product_df = (
    df.groupby('Product Name')
      .agg(
         OrderCount   = ('Order ID', 'nunique'),
         TotalQuantity= ('Quantity', 'sum'),
         TotalSales   = ('Sales', 'sum'),
         TotalProfit  = ('Profit', 'sum'),
         AvgPrice     = ('unit_price', 'mean')
      )
      .reset_index()
)

In [ ]:
# Chuẩn hóa dữ liệu product
X = product_df[['OrderCount', 'TotalQuantity', 'TotalSales', 'TotalProfit', 'AvgPrice']]
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [ ]:
# Silhouette scores for Products
prod_sil = []
for k in range(2,8):
    km = KMeans(n_clusters=k, random_state=42).fit(X_scaled)
    prod_sil.append(silhouette_score(X_scaled, km.labels_))
    sil = silhouette_score(X_scaled, km.labels_)
    print(f"Products - k={k}: Silhouette Score = {sil:.3f}")

# Plot silhouette analysis for RFM
plt.figure()
plt.plot(range(2,8), prod_sil, 'o-')
plt.xlabel('k clusters')
plt.ylabel('Silhouette Score')
plt.title('Silhouette Analysis cho RFM')
plt.show()

In [ ]:
# Chọn k=4 để tối ưu Silhouette Score lớn nhất và phân cụm bằng 4 vì có nhiều cụm đặc trưng hơn nhiều giá trị phân tích hơn tránh việc phân cụm (k=2) tổng quát
k_opt_p = 4
kmeans = KMeans(n_clusters=k_opt_p, random_state=42)
product_df['KMeans_Cluster'] = kmeans.fit_predict(X_scaled)
# Silhouette Score products
print("Silhouette Score products:", silhouette_score(X_scaled, product_df['KMeans_Cluster']))

In [ ]:
# Thống kê trung bình theo cụm
cluster_summary = product_df.groupby('KMeans_Cluster')[['OrderCount', 'TotalQuantity', 'TotalSales', 'TotalProfit', 'AvgPrice']].mean().round(2)
print(" Trung bình theo cụm sản phẩm:")
print(cluster_summary)
# Số lượng sản phẩm trong mỗi cụm
counts = product_df['KMeans_Cluster'].value_counts().sort_index()
print("\n Số lượng sản phẩm trong mỗi cụm:")
print(counts)

In [ ]:
# Vẽ biểu đồ trực quan
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
pcs = pca.fit_transform(X_scaled)
df_pca = pd.DataFrame(pcs, columns=['PC1','PC2'])
df_pca['KMeans_Cluster'] = product_df['KMeans_Cluster']
sns.scatterplot(data=df_pca, x='PC1', y='PC2', hue='KMeans_Cluster', palette='tab10')
#PCA
plt.title("PCA projection of Product Clusters")
plt.show()
#pairplot
sns.pairplot(product_df, hue='KMeans_Cluster', vars=['OrderCount', 'TotalQuantity', 'TotalSales', 'TotalProfit', 'AvgPrice'], palette='tab10')
plt.suptitle("Phân cụm sản phẩm theo RFM", y=1.02)
plt.show()

# **DBSCAN ( Phân cụm khách hàng )**

In [ ]:
from sklearn.cluster import DBSCAN
from sklearn.metrics import silhouette_score
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import seaborn as sns

# --- DBSCAN clustering ---
dbscan = DBSCAN(eps=0.33, min_samples=4)
rfm['DBSCAN_Cluster'] = dbscan.fit_predict(rfm_scaled)

# --- Thống kê cụm & outlier ---
n_total_clusters = len(set(rfm['DBSCAN_Cluster'])) - (1 if -1 in rfm['DBSCAN_Cluster'].values else 0)
n_outliers = (rfm['DBSCAN_Cluster'] == -1).sum()

print(f"Số lượng cụm (không tính noise): {n_total_clusters}")
print(f"Số khách hàng nhiễu (outliers): {n_outliers}")


In [ ]:
# --- Tính silhouette score (chỉ tính điểm hợp lệ) ---
if n_total_clusters > 1:
    mask = rfm['DBSCAN_Cluster'] != -1
    score = silhouette_score(rfm_scaled[mask], rfm['DBSCAN_Cluster'][mask])
    print(f"Silhouette Score (loại bỏ noise): {score:.3f}")
else:
    print("Không thể tính Silhouette Score (chỉ có 1 cụm hoặc toàn noise)")
    score = None

# --- PCA trực quan hóa ---
pca = PCA(n_components=2)
rfm_pca = pca.fit_transform(rfm_scaled)
rfm['PCA1'] = rfm_pca[:, 0]
rfm['PCA2'] = rfm_pca[:, 1]

plt.figure(figsize=(10, 6))
sns.scatterplot(data=rfm, x='PCA1', y='PCA2', hue='DBSCAN_Cluster', palette='Set1', s=60)
plt.title("Phân cụm khách hàng bằng DBSCAN (PCA 2D)")
plt.legend(title='Cụm', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

# **DBSCAN ( Phân cụm sản phẩm )**

In [ ]:
from sklearn.cluster import DBSCAN
from sklearn.metrics import silhouette_score
from sklearn.decomposition import PCA
import seaborn as sns
import matplotlib.pyplot as plt

# --- Áp dụng DBSCAN ---
dbscan = DBSCAN(eps=2.5, min_samples=4)
product_df['DBSCAN_Cluster'] = dbscan.fit_predict(X_scaled)

# --- Thống kê cụm & outlier ---
n_clusters = len(set(product_df['DBSCAN_Cluster'])) - (1 if -1 in product_df['DBSCAN_Cluster'] else 0)
n_noise = (product_df['DBSCAN_Cluster'] == -1).sum()

print(f"Số lượng cụm (không tính noise): {n_clusters}")
print(f"Số sản phẩm nhiễu (noise): {n_noise}")

In [ ]:
# --- Tính silhouette score (chỉ tính nếu có trên 1 cụm và loại noise) ---
if n_clusters > 1:
    mask = product_df['DBSCAN_Cluster'] != -1
    score = silhouette_score(X_scaled[mask], product_df['DBSCAN_Cluster'][mask])
    print(f"Silhouette Score (loại bỏ noise): {score:.3f}")
else:
    print("Không thể tính Silhouette Score (chỉ có 1 cụm hoặc toàn noise)")
    score = None

# --- PCA trực quan hóa ---
pca = PCA(n_components=2)
product_pca = pca.fit_transform(X_scaled)
product_df['PCA1'] = product_pca[:, 0]
product_df['PCA2'] = product_pca[:, 1]

plt.figure(figsize=(10,6))
sns.scatterplot(data=product_df, x='PCA1', y='PCA2', hue='DBSCAN_Cluster', palette='Set1', s=60)
plt.title("Phân cụm sản phẩm bằng DBSCAN (PCA 2D)")
plt.legend(title='Cụm', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

In [ ]:
#Kết quả tổng hợp trung bình của từng cụm sản phẩm
product_summary = product_df.groupby('DBSCAN_Cluster')[['OrderCount', 'TotalQuantity', 'TotalSales', 'TotalProfit', 'AvgPrice']].mean().round(2)
print("Kết quả tổng hợp trung bình của từng cụm sản phẩm:")
print(product_summary)
#Số lượng sản phẩm trong mỗi cụm
counts = product_df['DBSCAN_Cluster'].value_counts().sort_index()
print("\nSố lượng sản phẩm trong mỗi cụm:")
print(counts)

# **So sánh K-means và DBSCAN**

In [ ]:
from sklearn.cluster import KMeans, DBSCAN
from sklearn.metrics import silhouette_score
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import matplotlib.ticker as mtick

# 1. KMeans clustering
def apply_kmeans(X_scaled, df, n_clusters, label_name):
    km = KMeans(n_clusters=n_clusters, random_state=42)
    labels = km.fit_predict(X_scaled)
    df[f"{label_name}_KMeans"] = labels
    score = silhouette_score(X_scaled, labels)
    print(f"[KMeans] {label_name} – k={n_clusters}: Silhouette Score = {score:.3f}")
    return score

# 2. DBSCAN clustering
def apply_dbscan(X_scaled, df, eps, min_samples, label_name):
    db = DBSCAN(eps=eps, min_samples=min_samples)
    labels = db.fit_predict(X_scaled)
    df[f"{label_name}_DBSCAN"] = labels
    n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
    if n_clusters > 1:
        score = silhouette_score(X_scaled[labels != -1], labels[labels != -1])
        print(f"[DBSCAN] {label_name} – eps={eps}: Silhouette Score = {score:.3f}")
    else:
        score = None
        print(f"[DBSCAN] {label_name} – Không đủ cụm hợp lệ để tính Silhouette Score.")
    return score

# 3. So sánh PCA trực quan hóa
def plot_pca_comparison(X_scaled, df, label_name, kmeans_k=None):
    pca = PCA(n_components=2)
    pcs = pca.fit_transform(X_scaled)
    df_pca = pd.DataFrame(pcs, columns=['PC1','PC2'])
    df_pca[label_name+'_KMeans'] = df[f"{label_name}_KMeans"]
    df_pca[label_name+'_DBSCAN'] = df[f"{label_name}_DBSCAN"]

    fig, axes = plt.subplots(1, 2, figsize=(12, 5))
    sns.scatterplot(ax=axes[0], data=df_pca, x='PC1', y='PC2',
                    hue=label_name+'_KMeans', palette='tab10', s=50)
    axes[0].set_title(f"{label_name} – KMeans (k={kmeans_k})")

    sns.scatterplot(ax=axes[1], data=df_pca, x='PC1', y='PC2',
                    hue=label_name+'_DBSCAN', palette='Set1', s=50)
    axes[1].set_title(f"{label_name} – DBSCAN")
    plt.tight_layout()
    plt.show()

# 4. Heatmap RFM trung bình theo cụm
def plot_rfm_heatmap(rfm_df, cluster_col):
    rfm_group = rfm_df.groupby(cluster_col)[['Recency', 'Frequency', 'Monetary']].mean().round(2)
    plt.figure(figsize=(8,5))
    sns.heatmap(rfm_group, annot=True, cmap='YlGnBu', fmt='.1f')
    plt.title("Trung bình RFM theo cụm")
    plt.show()

# 5. Đếm số lượng khách hàng theo cụm
def plot_cluster_distribution(df, cluster_col, title="Phân phối khách hàng theo cụm"):
    plt.figure(figsize=(6,4))
    sns.countplot(data=df, x=cluster_col, palette='Set2')
    plt.title(title)
    plt.xlabel("Cụm")
    plt.ylabel("Số lượng")
    plt.show()

# 6. Tỷ lệ phần trăm khách hàng theo cụm
def plot_cluster_percentage(df, cluster_col):
    cluster_pct = df[cluster_col].value_counts(normalize=True).sort_index() * 100
    cluster_pct = cluster_pct.round(1)

    plt.figure(figsize=(6,4))
    ax = sns.barplot(x=cluster_pct.index, y=cluster_pct.values, palette='pastel')
    ax.yaxis.set_major_formatter(mtick.PercentFormatter())
    plt.title("Tỷ lệ phần trăm khách hàng theo cụm")
    plt.ylabel("Tỷ lệ (%)")
    plt.xlabel("Cụm")
    plt.show()


In [ ]:
# --- Phân cụm RFM (Khách hàng) ---
rfm_kmeans_score = apply_kmeans(rfm_scaled, rfm, n_clusters=3, label_name='RFM')
rfm_dbscan_score = apply_dbscan(rfm_scaled, rfm, eps=0.33, min_samples=4, label_name='RFM')
plot_pca_comparison(rfm_scaled, rfm, label_name='RFM', kmeans_k=3)
plot_rfm_heatmap(rfm, cluster_col='RFM_KMeans')
plot_cluster_distribution(rfm, cluster_col='RFM_KMeans')
plot_cluster_percentage(rfm, cluster_col='RFM_KMeans')

# --- Phân cụm Product (Sản phẩm) ---
prod_kmeans_score = apply_kmeans(X_scaled, product_df, n_clusters=4, label_name='Prod')
prod_dbscan_score = apply_dbscan(X_scaled, product_df, eps=2.5, min_samples=4, label_name='Prod')
plot_pca_comparison(X_scaled, product_df, label_name='Prod', kmeans_k=4)
